In [2]:
import numpy as np
import random, math
import copy, sys

A = np.array([[ 16,  8,  2], [ 24, -1024,  48], [ 96, -32,  16]])
b = np.array([0.02050781, 0.55859375, 0.1640625 ])
# x = np.array([3/2048, -1/2048, 1/2048])
# xi = 1/1024*qi1 + 1/2048*qi2 - 1/1024*qi3 - 1/2048qi4

[Dimension, ny] = A.shape
qubits = 2
dcm = 10

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,-2*(l+dcm))*pow(A[k][i],2)
            cef2 = pow(2,-(l+dcm)+1)*A[k][i]*b[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, -(l1+dcm)-(l2+dcm)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, -(l1+dcm)-(l2+dcm)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = i + 1
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = i + 1
            qdrt2 = j + 1
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")      


# Matrix Q is
[[-4.80041503e-02  9.58251953e-03  0.00000000e+00  0.00000000e+00
  -5.24902344e-02 -2.62451172e-02  5.24902344e-02  2.62451172e-02
   5.18798828e-03  2.59399414e-03 -5.18798828e-03 -2.59399414e-03]
 [ 0.00000000e+00 -2.63977050e-02  0.00000000e+00  0.00000000e+00
  -2.62451172e-02 -1.31225586e-02  2.62451172e-02  1.31225586e-02
   2.59399414e-03  1.29699707e-03 -2.59399414e-03 -1.29699707e-03]
 [ 0.00000000e+00  0.00000000e+00  6.71691894e-02  9.58251953e-03
   5.24902344e-02  2.62451172e-02 -5.24902344e-02 -2.62451172e-02
  -5.18798828e-03 -2.59399414e-03  5.18798828e-03  2.59399414e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.11889648e-02
   2.62451172e-02  1.31225586e-02 -2.62451172e-02 -1.31225586e-02
  -2.59399414e-03 -1.29699707e-03  2.59399414e-03  1.29699707e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   2.12815857e+00  1.00103760e+00  0.00000000e+00  0.00000000e+00
  -9.46960449e-02 -4.73480225e-02  9.46960449e-02  4.73480

In [3]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-0.0480041503125, ('q2','q2'):-0.0263977050390625, ('q3','q3'):0.067169189375, ('q4','q4'):0.0311889648046875, ('q5','q5'):2.128158569375, ('q6','q6'):0.8138198852734375, ('q7','q7'):-0.12608337406250003, ('q8','q8'):-0.3133010864453125, ('q9','q9'):-0.055130004873046874, ('q10','q10'):-0.028176307673339843, ('q11','q11'):0.060020446767578124, ('q12','q12'):0.029398918146972656}

quadratic = {('q1','q2'):0.00958251953125, ('q1','q5'):-0.052490234375, ('q1','q6'):-0.0262451171875, ('q1','q7'):0.052490234375, ('q1','q8'):0.0262451171875, ('q1','q9'):0.00518798828125, ('q1','q10'):0.002593994140625, ('q1','q11'):-0.00518798828125, ('q1','q12'):-0.002593994140625, ('q2','q5'):-0.0262451171875, ('q2','q6'):-0.01312255859375, ('q2','q7'):0.0262451171875, ('q2','q8'):0.01312255859375, ('q2','q9'):0.002593994140625, ('q2','q10'):0.0012969970703125, ('q2','q11'):-0.002593994140625, ('q2','q12'):-0.0012969970703125, ('q3','q4'):0.00958251953125, ('q3','q5'):0.052490234375, ('q3','q6'):0.0262451171875, ('q3','q7'):-0.052490234375, ('q3','q8'):-0.0262451171875, ('q3','q9'):-0.00518798828125, ('q3','q10'):-0.002593994140625, ('q3','q11'):0.00518798828125, ('q3','q12'):0.002593994140625, ('q4','q5'):0.0262451171875, ('q4','q6'):0.01312255859375, ('q4','q7'):-0.0262451171875, ('q4','q8'):-0.01312255859375, ('q4','q9'):-0.002593994140625, ('q4','q10'):-0.0012969970703125, ('q4','q11'):0.002593994140625, ('q4','q12'):0.0012969970703125, ('q5','q6'):1.00103759765625, ('q5','q9'):-0.094696044921875, ('q5','q10'):-0.0473480224609375, ('q5','q11'):0.094696044921875, ('q5','q12'):0.0473480224609375, ('q6','q9'):-0.0473480224609375, ('q6','q10'):-0.02367401123046875, ('q6','q11'):0.0473480224609375, ('q6','q12'):0.02367401123046875, ('q7','q8'):1.00103759765625, ('q7','q9'):0.094696044921875, ('q7','q10'):0.0473480224609375, ('q7','q11'):-0.094696044921875, ('q7','q12'):-0.0473480224609375, ('q8','q9'):0.0473480224609375, ('q8','q10'):0.02367401123046875, ('q8','q11'):-0.0473480224609375, ('q8','q12'):-0.02367401123046875, ('q9','q10'):0.002445220947265625, ('q11','q12'):0.002445220947265625}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9    energy num_oc. chain_b.
0    1   1   0   0  1  0  0  0  0  0  1  0 -0.339364      12      0.0
1    1   0   0   0  1  0  0  0  0  0  1  0 -0.338753      27      0.0
2    1   0   0   0  1  0  0  0  0  0  1  1 -0.338753       8      0.0
3    1   0   0   0  0  0  0  0  0  0  1  1 -0.337654       5      0.0
4    1   1   0   1  1  0  0  0  0  0  1  0  -0.33753      21      0.0
5    1   1   0   0  0  0  0  0  0  0  1  1 -0.337117       6      0.0
6    1   1   0   0  0  0  0  0  0  0  1  0 -0.336968      18      0.0
7    1   0   0   1  1  0  0  0  0  0  1  0 -0.336919      31      0.0
8    1   0   0   1  1  0  0  0  0  0  1  1 -0.336919      11      0.0
9    1   1   0   0  1  0  0  0  0  0  1  1 -0.336919       5      0.0
10   1   1   0   1  1  0  0  0  0  0  1  1 -0.335085       6      0.0
11   1   0   0   0  0  0  0  0  0  0  1  0  -0.33506      18      0.0
12   1   0   0   1  0  0  0  0  0  0  1  1 -0.334523      10      0.0
13   1   1   1   0  

In [4]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9    energy num_oc. chain_b.
0    1   1   0   0  1  0  0  0  0  0  1  0 -0.339364       4      0.0
1    1   0   0   0  1  0  0  0  0  0  1  0 -0.338753       4      0.0
2    1   0   0   0  1  0  0  0  0  0  1  1 -0.338753       1      0.0
3    1   0   0   0  0  0  0  0  0  0  1  1 -0.337654      10      0.0
4    1   1   0   1  1  0  0  0  0  0  1  0  -0.33753       3      0.0
5    1   1   0   0  0  0  0  0  0  0  1  1 -0.337117       4      0.0
6    1   1   0   0  0  0  0  0  0  0  1  0 -0.336968       8      0.0
7    1   0   0   1  1  0  0  0  0  0  1  0 -0.336919       3      0.0
8    1   0   0   1  1  0  0  0  0  0  1  1 -0.336919       4      0.0
9    1   1   0   0  1  0  0  0  0  0  1  1 -0.336919       2      0.0
10   1   1   0   1  1  0  0  0  0  0  1  1 -0.335085       8      0.0
11   1   0   0   0  0  0  0  0  0  0  1  0  -0.33506       5      0.0
12   1   0   0   1  0  0  0  0  0  0  1  1 -0.334523       8      0.0
13   1   1   1   0  

In [5]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9    energy num_oc. chain_b.
0    1   1   0   0  1  0  0  0  0  0  1  0 -0.339364       9      0.0
1    1   0   0   0  1  0  0  0  0  0  1  0 -0.338753       4      0.0
2    1   0   0   0  1  0  0  0  0  0  1  1 -0.338753       4      0.0
3    1   0   0   0  0  0  0  0  0  0  1  1 -0.337654       3      0.0
4    1   1   0   1  1  0  0  0  0  0  1  0  -0.33753      18      0.0
5    1   1   0   0  0  0  0  0  0  0  1  0 -0.336968       9      0.0
6    1   0   0   1  1  0  0  0  0  0  1  0 -0.336919      17      0.0
7    1   0   0   1  1  0  0  0  0  0  1  1 -0.336919      10      0.0
8    1   1   0   0  1  0  0  0  0  0  1  1 -0.336919      13      0.0
9    1   1   0   1  1  0  0  0  0  0  1  1 -0.335085      17      0.0
10   1   0   0   0  0  0  0  0  0  0  1  0  -0.33506       6      0.0
11   1   0   0   1  0  0  0  0  0  0  1  1 -0.334523       5      0.0
12   1   1   1   0  1  0  0  0  0  0  1  0 -0.334474       7      0.0
13   1   1   0   1  

In [6]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9    energy num_oc. chain_b.
0    1   1   0   0  1  0  0  0  0  0  1  0 -0.339364       3      0.0
1    1   0   0   0  1  0  0  0  0  0  1  0 -0.338753       7      0.0
2    1   0   0   0  1  0  0  0  0  0  1  1 -0.338753      11      0.0
3    1   0   0   0  0  0  0  0  0  0  1  1 -0.337654      10      0.0
4    1   1   0   1  1  0  0  0  0  0  1  0  -0.33753       6      0.0
5    1   1   0   0  0  0  0  0  0  0  1  1 -0.337117       5      0.0
6    1   1   0   0  0  0  0  0  0  0  1  0 -0.336968       5      0.0
7    1   0   0   1  1  0  0  0  0  0  1  0 -0.336919      10      0.0
8    1   0   0   1  1  0  0  0  0  0  1  1 -0.336919      12      0.0
9    1   1   0   0  1  0  0  0  0  0  1  1 -0.336919       6      0.0
10   1   1   0   1  1  0  0  0  0  0  1  1 -0.335085       3      0.0
11   1   0   0   0  0  0  0  0  0  0  1  0  -0.33506       5      0.0
12   1   0   0   1  0  0  0  0  0  0  1  1 -0.334523       5      0.0
13   1   1   1   0  

In [7]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9    energy num_oc. chain_b.
0    1   1   0   0  1  0  0  0  0  0  1  0 -0.339364       3      0.0
1    1   0   0   0  1  0  0  0  0  0  1  0 -0.338753       7      0.0
2    1   0   0   0  1  0  0  0  0  0  1  1 -0.338753       3      0.0
3    1   0   0   0  0  0  0  0  0  0  1  1 -0.337654       5      0.0
4    1   1   0   1  1  0  0  0  0  0  1  0  -0.33753       6      0.0
5    1   1   0   0  0  0  0  0  0  0  1  1 -0.337117       6      0.0
6    1   1   0   0  0  0  0  0  0  0  1  0 -0.336968       3      0.0
7    1   0   0   1  1  0  0  0  0  0  1  0 -0.336919      12      0.0
8    1   0   0   1  1  0  0  0  0  0  1  1 -0.336919       5      0.0
9    1   1   0   0  1  0  0  0  0  0  1  1 -0.336919       7      0.0
10   1   1   0   1  1  0  0  0  0  0  1  1 -0.335085      13      0.0
11   1   0   0   0  0  0  0  0  0  0  1  0  -0.33506       6      0.0
12   1   0   0   1  0  0  0  0  0  0  1  1 -0.334523       9      0.0
13   1   1   1   0  

In [8]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=10000)
print(sampleset)

    q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9    energy num_oc. chain_b.
0    1   1   0   0  1  0  0  0  0  0  1  0 -0.339364      24      0.0
461  1   1   0   0  1  0  0  0  0  0  1  0 -0.339364       1 0.083333
1    1   0   0   0  1  0  0  0  0  0  1  0 -0.338753      29      0.0
2    1   0   0   0  1  0  0  0  0  0  1  1 -0.338753      75      0.0
377  1   0   0   0  1  0  0  0  0  0  1  1 -0.338753       1 0.083333
3    1   0   0   0  0  0  0  0  0  0  1  1 -0.337654     193      0.0
452  1   0   0   0  0  0  0  0  0  0  1  1 -0.337654       1 0.083333
454  1   0   0   0  0  0  0  0  0  0  1  1 -0.337654       1 0.083333
463  1   0   0   0  0  0  0  0  0  0  1  1 -0.337654       1 0.083333
482  1   0   0   0  0  0  0  0  0  0  1  1 -0.337654       1 0.083333
4    1   1   0   1  1  0  0  0  0  0  1  0  -0.33753      43      0.0
5    1   1   0   0  0  0  0  0  0  0  1  1 -0.337117     164      0.0
343  1   1   0   0  0  0  0  0  0  0  1  1 -0.337117       1 0.083333
6    1   1   0   0  

In [9]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=10000)
print(sampleset)

    q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9    energy num_oc. chain_b.
0    1   1   0   0  1  0  0  0  0  0  1  0 -0.339364      58      0.0
389  1   1   0   0  1  0  0  0  0  0  1  0 -0.339364       1 0.083333
1    1   0   0   0  1  0  0  0  0  0  1  0 -0.338753     105      0.0
2    1   0   0   0  1  0  0  0  0  0  1  1 -0.338753      39      0.0
3    1   0   0   0  0  0  0  0  0  0  1  1 -0.337654      35      0.0
4    1   1   0   1  1  0  0  0  0  0  1  0  -0.33753      50      0.0
5    1   1   0   0  0  0  0  0  0  0  1  1 -0.337117      15      0.0
6    1   1   0   0  0  0  0  0  0  0  1  0 -0.336968      31      0.0
7    1   0   0   1  1  0  0  0  0  0  1  0 -0.336919      79      0.0
429  1   0   0   1  1  0  0  0  0  0  1  0 -0.336919       1 0.083333
8    1   0   0   1  1  0  0  0  0  0  1  1 -0.336919      55      0.0
9    1   1   0   0  1  0  0  0  0  0  1  1 -0.336919      34      0.0
10   1   1   0   1  1  0  0  0  0  0  1  1 -0.335085      24      0.0
11   1   0   0   0  

In [10]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=10000)
print(sampleset)

    q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9    energy num_oc. chain_b.
0    1   1   0   0  1  0  0  0  0  0  1  0 -0.339364     131      0.0
1    1   0   0   0  1  0  0  0  0  0  1  0 -0.338753     141      0.0
2    1   0   0   0  1  0  0  0  0  0  1  1 -0.338753     105      0.0
469  1   0   0   0  1  0  0  0  0  0  1  1 -0.338753       1 0.083333
3    1   0   0   0  0  0  0  0  0  0  1  1 -0.337654      89      0.0
490  1   0   0   0  0  0  0  0  0  0  1  1 -0.337654       1 0.083333
4    1   1   0   1  1  0  0  0  0  0  1  0  -0.33753      97      0.0
5    1   1   0   0  0  0  0  0  0  0  1  1 -0.337117      71      0.0
504  1   1   0   0  0  0  0  0  0  0  1  1 -0.337117       1 0.083333
6    1   1   0   0  0  0  0  0  0  0  1  0 -0.336968      77      0.0
480  1   1   0   0  0  0  0  0  0  0  1  0 -0.336968       1 0.083333
7    1   0   0   1  1  0  0  0  0  0  1  0 -0.336919     100      0.0
8    1   0   0   1  1  0  0  0  0  0  1  1 -0.336919      92      0.0
9    1   1   0   0  